# Feature importance example

In [1]:
"""Feature importance example."""

import pprint

import more_itertools
import numpy as np
import pandas as pd

from skrough.chaos_measures import entropy, gini_impurity
from skrough.chaos_score import get_chaos_score_for_data
from skrough.dataprep import prepare_factorized_data
from skrough.feature_importance import get_feature_importance

## Dataset

Let's prepare a sample data set - "Play Golf Dataset".

In [2]:
df = pd.DataFrame(
    np.array(
        [
            ["sunny", "hot", "high", "weak", "no"],
            ["sunny", "hot", "high", "strong", "no"],
            ["overcast", "hot", "high", "weak", "yes"],
            ["rain", "mild", "high", "weak", "yes"],
            ["rain", "cool", "normal", "weak", "yes"],
            ["rain", "cool", "normal", "strong", "no"],
            ["overcast", "cool", "normal", "strong", "yes"],
            ["sunny", "mild", "high", "weak", "no"],
            ["sunny", "cool", "normal", "weak", "yes"],
            ["rain", "mild", "normal", "weak", "yes"],
            ["sunny", "mild", "normal", "strong", "yes"],
            ["overcast", "mild", "high", "strong", "yes"],
            ["overcast", "hot", "normal", "weak", "yes"],
            ["rain", "mild", "high", "strong", "no"],
        ],
        dtype=object,
    ),
    columns=["Outlook", "Temperature", "Humidity", "Wind", "Play"],
)
TARGET_COLUMN = "Play"
df

,Outlook,Temperature,Humidity,Wind,Play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


## Prepare data

Factorize dataset and obtain the sizes of feature domains.

In [3]:
x, x_domain_sizes, y, y_domain_size = prepare_factorized_data(df, TARGET_COLUMN)
column_names = np.array([col for col in df.columns if col != TARGET_COLUMN])

print("Conditional data:")
print(x)
print()
print("Conditional data feature domain sizes:")
print(x_domain_sizes)
print()
print("Target data:")
print(y)
print()
print("Target data feature domain size:")
print(y_domain_size)

Conditional data:
[[0 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [2 1 0 0]
 [2 2 1 0]
 [2 2 1 1]
 [1 2 1 1]
 [0 1 0 0]
 [0 2 1 0]
 [2 1 1 0]
 [0 1 1 1]
 [1 1 0 1]
 [1 0 1 0]
 [2 1 0 1]]

Conditional data feature domain sizes:
[3 3 2 2]

Target data:
[0 0 1 1 1 0 1 0 1 1 1 1 1 0]

Target data feature domain size:
2


## Measure of disorder in the dataset - chaos score

In the context of the given dataset, a chaos score is a quantity that characterizes
a subset of features and, more or less, presents the disorder of decisions in
the equivalence classes induced by the subsets of features.

In most cases it is reasonable to assume that the chaos score function is monotonic
with respect to subset relation, i.e., for subsets of features $A \subseteq B$,
the chaos score for $A$ should be less or equal to that for $B$.

Attributes are given by their ordinal numbers.

Let's try two standard approaches, i.e., gini_impurity and entropy.

In [4]:
for chaos_function in [gini_impurity, entropy]:
    print(chaos_function.__name__)
    for attrs in [[0], [0, 1], [0, 1, 3], [0, 1, 2, 3]]:
        print(
            f"chaos score for attrs {attrs}({column_names[attrs]}) = ",
            get_chaos_score_for_data(
                x=x,
                x_counts=x_domain_sizes,
                y=y,
                y_count=y_domain_size,
                chaos_fun=chaos_function,
                attrs=attrs,
            ),
        )
    print()
    print()

gini_impurity
chaos score for attrs [0](['Outlook']) =  0.34285714285714286
chaos score for attrs [0, 1](['Outlook' 'Temperature']) =  0.23809523809523808
chaos score for attrs [0, 1, 3](['Outlook' 'Temperature' 'Wind']) =  0.0
chaos score for attrs [0, 1, 2, 3](['Outlook' 'Temperature' 'Humidity' 'Wind']) =  0.0


entropy
chaos score for attrs [0](['Outlook']) =  0.6935361388961918
chaos score for attrs [0, 1](['Outlook' 'Temperature']) =  0.4824919644402477
chaos score for attrs [0, 1, 3](['Outlook' 'Temperature' 'Wind']) =  0.0
chaos score for attrs [0, 1, 2, 3](['Outlook' 'Temperature' 'Humidity' 'Wind']) =  0.0




## Assessing feature importance

We can use the above chaos score functions for assessing the features, i.e.,
we can observe the chaos score change if a given feature is removed.

To follow a more realistic example, we can use an enseble of feature subsets, i.e.,
a family of subsets of all atributes, and not just a single subset of features,
computing the total or average chaos score change over several possible appearances
of the attribute in the ensemble elements.

In [5]:
attr_subset_ensemble = [
    [[0, 2], [0, 3], [0], [2, 3], [1, 2, 3]],
    [[0], [0, 1], [1, 2]],
    [list(elem) for elem in more_itertools.powerset(range(4))],
]
for chaos_function in [gini_impurity, entropy]:
    print(chaos_function.__name__)
    for attr_subset in attr_subset_ensemble:
        print("feature importance for attribute subset ensemble: ")
        pprint.pprint(attr_subset, compact=True)
        print(
            get_feature_importance(
                x,
                x_domain_sizes,
                y,
                y_domain_size,
                column_names,
                attr_subset,
                chaos_fun=chaos_function,
            )
        )
        print()
    print()
    print()

gini_impurity
feature importance for attribute subset ensemble: 
[[0, 2], [0, 3], [0], [2, 3], [1, 2, 3]]
        column  count  total_gain  avg_gain
0      Outlook    3.0    0.578912  0.192971
1  Temperature    1.0    0.047619  0.047619
2     Humidity    3.0    0.342857  0.114286
3         Wind    3.0    0.269728  0.089909

feature importance for attribute subset ensemble: 
[[0], [0, 1], [1, 2]]
        column  count  total_gain  avg_gain
0      Outlook    2.0    0.318707  0.159354
1  Temperature    2.0    0.126871  0.063435
2     Humidity    1.0    0.095238  0.095238
3         Wind    0.0    0.000000  0.000000

feature importance for attribute subset ensemble: 
[[], [0], [1], [2], [3], [0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3],
 [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3], [0, 1, 2, 3]]
        column  count  total_gain  avg_gain
0      Outlook    8.0    1.959864  0.244983
1  Temperature    8.0    0.455102  0.056888
2     Humidity    8.0    0.791837  0.098980
3         Wind    8.